## Agents Env


### Agent Workspace Environment


Each agent work in ubuntu environment:

```dockerfile
FROM ubuntu:latest

RUN apt-get update

RUN apt install git -y

RUN apt install python3 -y

RUN apt install python3-pip -y

RUN mkdir -p /agent_workspace

WORKDIR /agent_workspace

CMD ["/bin/bash"]
```

Create docker image:

```bash
docker build -t agent_env .
```


Basic prompt for agents:

````python

prompt = """Act as {role}. Your responsibility is {responsibility}.

You have access to an Ubuntu environment, previously updated and located in your work folder.```
````


### Create Agent Env


In [1]:
import docker
import time

client = docker.from_env()
container_ref = None
CONTAINER_IMAGE = "agent_env"

def create_container(container_name):
    global container_ref

    existing_containers = client.containers.list(
        all=True, filters={"name": CONTAINER_NAME}
    )
    if existing_containers:
        container = existing_containers[0]
        if container.status != "running":
            container.start()
            time.sleep(20)
        container.reload()
        container_ref = container
        return "Container criado com sucesso."

    container = client.containers.run(
        CONTAINER_IMAGE,
        name=container_name,
        detach=True,
        tty=True,
        stdin_open=True,
        working_dir="/agent_workspace",
        command="/bin/bash",
    )

    time.sleep(20)

    container_ref = container
    return "Container criado com sucesso."

CONTAINER_NAME = "qa_agent_container"
create_container(CONTAINER_NAME)

'Container criado com sucesso.'

In [ ]:
def delete_container(container_name):
    existing_containers = client.containers.list(
        all=True, filters={"name": container_name}
    )

    if existing_containers:
        container = existing_containers[0]
        container.stop()
        container.remove()
        return "Container deletado com sucesso."


# CONTAINER_NAME = "qa_agent_container"
# delete_container(CONTAINER_NAME)

In [2]:
def get_container():
    global container_ref

    if container_ref:
        container_ref.reload()
        if container_ref.status == "running":
            return container_ref

    containers = client.containers.list(all=True, filters={"name": CONTAINER_NAME})
    if containers:
        container = containers[0]
        if container.status != "running":
            container.start()
            time.sleep(60)
        container.reload()
        container_ref = container
        return container

    return None

### Utils function (get_container)


In [3]:
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama

# from langgraph.checkpoint.memory import MemorySaver

In [4]:
model_name = "llama3.1"
# model_name = "mistral-nemo"

model = ChatOllama(model=model_name)

### Run Command Tool (basic tool):


In [5]:
@tool
def terminal(command: str) -> str:
    """
    Execute a shell command to be executed.
    """
    container = get_container()
    if not container:
        return "No container found. Please try again."

    try:
        result = container.exec_run(f"/bin/sh -c '{command}'")
        return result.output.decode("utf-8")

    except Exception as e:
        return f"Error executing command: {str(e)}"

In [ ]:
# array of tools available for the agent
tools = [terminal]

# create the agent executor
agent_executor = create_react_agent(
    name="qa_agent",
    tools=tools,
    model=model,
)

# agent description
description = """You are a Software Quality Analyst (QA) responsible for setting up and preparing the project environment.  
Follow the instructions carefully and execute the tasks precisely.  

System Information:
- OS: Ubuntu (root access available).
- Shell: Bash (no interactive commands allowed)."""

In [16]:
planning_prompt = ChatPromptTemplate.from_messages([
    ("system", description),
    ("user", """You are an expert in software analysis and diagnostics. Your task is to analyze the following request and generate a detailed execution plan to accomplish it.

Task:
{task}

Your execution plan should include:
1. Steps to retrieve and set up the repository.
2. Instructions to run the unit tests and capture failing test cases.
3. Guidelines for diagnosing the failed tests and identifying potential issues.
4. Recommendations for improving the code based on the test failures.
5. A structured outline for compiling the final technical report.

Ensure the plan is clear, logically structured, and executable by a technical professional.""" )
])

In [18]:
task = """Analyze the repository https://github.com/astropy/astropy.git by checking out the exact commit d16bfe05a744909de4b27f5875fe0d4ed41ce607. Generate a technical report that includes a diagnosis and suggestions for improvement.

Your analysis should include:
- Running the unit tests and identifying failed test cases.
- Examining the cause of failures and diagnosing potential issues.
- Proposing improvements based on the identified problems.
- Structuring the findings into a clear and concise report."""

planning_response = model.invoke(planning_prompt.invoke({"task": task}))

print("Planned Tasks:\n", planning_response.content)

Planned Tasks:
 **Technical Execution Plan: Analyzing Repository https://github.com/astropy/astropy.git**

**Step 1: Retrieve and Setup the Repository (Estimated Time: 15 minutes)**

1. **Clone the repository**: Using Bash, execute:
   ```bash
git clone https://github.com/astropy/astropy.git
```
   This command clones the Astropy repository onto your local machine.

2. **Navigate to the cloned repository**: Execute:
   ```
cd astropy
```

3. **Checkout the exact commit d16bfe05a744909de4b27f5875fe0d4ed41ce607**:
   ```bash
git checkout d16bfe05a744909de4b27f5875fe0d4ed41ce607
```
   This command checks out the exact commit specified in the task, ensuring we're working with a stable and reproducible environment.

**Step 2: Run Unit Tests (Estimated Time: 30 minutes)**

1. **Navigate to the tests directory**: Execute:
   ```
cd tests/
```

2. **Run the unit tests using the `py.test` command**: Execute:
   ```bash
pytest --junit-xml=astropy_test_results.xml -v
```
   This command runs all

In [21]:
execution_prompt = ChatPromptTemplate.from_messages([
    ("system", description),
    ("user", "Execute the following command and return the result:\n\n{task}")
])

In [22]:
task = """**Step 1: Retrieve and Setup the Repository (Estimated Time: 15 minutes)**

1. **Clone the repository**: Using Bash, execute:
   ```bash
git clone https://github.com/astropy/astropy.git
```
   This command clones the Astropy repository onto your local machine.

2. **Navigate to the cloned repository**: Execute:
   ```
cd astropy
```

3. **Checkout the exact commit d16bfe05a744909de4b27f5875fe0d4ed41ce607**:
   ```bash
git checkout d16bfe05a744909de4b27f5875fe0d4ed41ce607
```
   This command checks out the exact commit specified in the task, ensuring we're working with a stable and reproducible environment."""

prompt = execution_prompt.invoke({"task": task})

response = agent_executor.invoke(prompt, {"recursion_limit": 100})
print(response)

{'messages': [SystemMessage(content='You are a Software Quality Analyst (QA) responsible for setting up and preparing the project environment.  \nFollow the instructions carefully and execute the tasks precisely.  \n\nSystem Information:\n- OS: Ubuntu (root access available).\n- Shell: Bash (no interactive commands allowed).', additional_kwargs={}, response_metadata={}, id='2a765e54-153e-466b-b41a-c9d36f117511'), HumanMessage(content="Execute the following command and return the result:\n\n**Step 1: Retrieve and Setup the Repository (Estimated Time: 15 minutes)**\n\n1. **Clone the repository**: Using Bash, execute:\n   ```bash\ngit clone https://github.com/astropy/astropy.git\n```\n   This command clones the Astropy repository onto your local machine.\n\n2. **Navigate to the cloned repository**: Execute:\n   ```\ncd astropy\n```\n\n3. **Checkout the exact commit d16bfe05a744909de4b27f5875fe0d4ed41ce607**:\n   ```bash\ngit checkout d16bfe05a744909de4b27f5875fe0d4ed41ce607\n```\n   This 